# Gaussian Mixture Models Clustering

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Problem definition

$$\mathcal{N} (\mathbf{x} | \mathbf{\mu}^{(c)}, \Sigma^{(c)})= \frac{1}{(2\pi)^\frac{n}{2}\lvert{\Sigma^{(c)}}\rvert^\frac{1}{2}}e^{(-\frac{1}{2}(\mathbf{x} -\mathbf{\mu}^{(c)})^T\Sigma^{(c)-1}(\mathbf{x} - \mathbf{\mu}^{(c)}))}$$

<img src = "../figures/em2.png" height=400>

We shall also further define $\pi^{(c)}$, which is simply the probability of being in the cluster (think of it as $p(y)$).  Since $\pi$ is the probability of each cluster, it should sum to 1.

$$\quad 0 \leq \pi^{(c)} \leq 1 \quad ; \quad \sum\limits_{c=1}^k \pi^{(c)}=1$$

Then, using **Baye's theorem**, we can define the likelihood of a sample $\mathbf{x}^{(i)}$ belong to a cluster $c$ as simply:

$$p(y|x) = p(y) * p(x | y)$$
$$p(c | \mathbf{x}^{(i)}) = \pi^{(c)} * \mathcal{N} (\mathbf{x}^{(i)} | \mathbf{\mu}^{(c)}, \Sigma^{(c)}) $$

Finally, based on all the information, GMM defines its objective, i.e., to maximize

$$\prod \limits_{i=1}^m \sum_{c=1}^k \pi^{(c)} \mathcal{N} (\mathbf{x}^{(i)} | \mathbf{\mu}^{(c)}, \Sigma^{(c)})$$

## Derivatives

Our objective function is to find $\theta$ that maximize the log-likehood $\mathcal{L}$.  (Here we apply $\log$ for mathematical stability.)

$$\max_\theta \sum\limits_{i=1}^m \log \sum\limits_{c=1}^k \pi^{(c)} \mathcal{N}(\mathbf{x} | \mu^{(c)}, \Sigma^{(c)})$$

Our "normal" procedure would be to compute the gradient $\frac{d\mathcal{L}}{d\theta}$ and set it to 0.  however, if you try this yourself at home, you will find that it is not possible to find the closed form.

### Responsibilities $r$

Before anything, let us introduce a quantity that will play a central role in this algorithm: **responsibilities**.  We define the quantity

$$ r^{(i)}_{c} = \frac{\pi^{(c)}\mathcal{N}(\mathbf{x}^{(i)} \mid \mu^{(c)}, \Sigma^{(c)})}{\Sigma_{c=1}^{k} \pi^{(c)}\mathcal{N}(\mathbf{x}^{(i)} \mid \mu^{(c)}, \Sigma^{(c)})}$$ 

$r^{(i)}_{c}$ basically gives us $$ \frac{\text{Probability of $\mathbf{x}^{(i)}$ belonging to cluster c}}{\text{Probability of $\mathbf{x}^{(i)}$ over all clusters}} $$

Note that $r$ for a sample $i$ 

$$r^{(i)} = r^{(i)}_1, r^{(i)}_2, \cdots, r^{(i)}_k \in \mathbb{R}^k$$

$$\sum\limits_{c=1}^{k}r^{(i)}_c = 1$$

$$ r^{(i)}_c \geq 0 $$

### EM algorithm

Notice that $r$ actually depends on **mean, covariance and pi**.  But then, **mean, covariance, and pi** also depends on $r$.  Based on this, we can use EM algorithm, where we can (1) create a random mean, covariance, and pi, (2) calculate $r$, and then repeat 1 and 2 until certain stopping criteria.

### Total responsibility $N^{(c)}$

By summing all the total responsibility of the $c$ th cluster along all samples, we get $N^{(c)}$.

$$N_c = \sum\limits_{i=1}^{m}r^{(i)}_c \in \mathbb{R}^k $$

Note that this value does not necessarily sum to 1.

### Updating the mean

The update of the mean $\mu^{(c)}$ is given by:

$$ \mu_\text{new}^{(c)} = \frac{\sum\limits_{i=1}^{m}r^{(i)}_{c}\mathbf{x}^{(i)}}{\sum\limits_{i=1}^{m}r^{(i)}_{c}}$$

**Proof:**

We take a partial derivative of our objective function with respect to the mean parameters $\mu^{(c)}$. To simplify things, let's perform partial derivative without the log first and only consider one sample.

$$
	\frac{\partial p(\mathbf{x}^{(i)} | \theta)}{\partial \mu^{(c)}} = \sum\limits_{c=1}^k \pi^{(c)} \frac{\partial \mathcal{N}(\mathbf{x}^{(i)} | \mu^{(c)}, \Sigma^{(c)})}{\partial \mu^{(c)}} = \pi^{(c)} \frac{\partial \mathcal{N}(\mathbf{x}^{(i)} | \mu^{(c)}, \Sigma^{(c)})}{\partial \mu^{(c)}} = \pi^{(c)}(\mathbf{x}^{(i)} - \mu^{(c)})^T \Sigma^{(c)-1}\mathcal{N}(\mathbf{x}^{(i)} | \mu^{(c)}, \Sigma^{(c)})
$$

Now, applying log, since we know the partial derivative of $\log$ of $x$ is $\frac{1}{x}$, thus

$$
\frac{\partial \mathcal{L}}{\partial \mu^{(c)}} =\sum\limits_{i=1}^{m} \frac{\partial \log p(\mathbf{x}^{(i)} | \theta)}{\partial \mu^{(c)}} = \sum\limits_{i=1}^{m} \frac{1}{p(\mathbf{x}^{(i)} | \theta)} \frac{\partial p(\mathbf{x}^{(i)} | \theta) }{\partial \mu^{(c)}} = \sum\limits_{i=1}^{m}(\mathbf{x}^{(i)} - \mu^{(c)})^T \Sigma^{(c)-1}\frac{\pi^{(c)}\mathcal{N}(\mathbf{x}^{(i)} \mid \mu^{(c)}, \Sigma^{(c)})}{\Sigma_{c=1}^{k} \pi^{(c)}\mathcal{N}(\mathbf{x}^{(i)} \mid \mu^{(c)}, \Sigma^{(c)})}
$$

To simplify, we can substitute $r^{(i)}_{c}$ into the equation, thus

$$= \sum\limits_{i=1}^{m} r^{(i)}_{c}(\mathbf{x}^{(i)} - \mu^{(c)})^T\Sigma^{(c)-1}$$

We can now solve for $\mu^{(c)}$ so that $\frac{\partial \mathcal{L}}{\partial \mu^{(c)}} = 0$ and obtain

$$\sum\limits_{i=1}^{m} r^{(i)}_{c}(\mathbf{x}^{(i)} - \mu^{(c)})^T\Sigma^{(c)-1} = 0$$

Multiply both sides by $\Sigma$ will cancel out the inverse $\Sigma$, and move $\mu^{(c)}$ to another side

$$\sum\limits_{i=1}^{m} r^{(i)}_{c}\mathbf{x}^{(i)}  = \sum\limits_{i=1}^{m} r^{(i)}_{c}\mu^{(c)}$$

$$\frac{\sum\limits_{i=1}^{m} r^{(i)}_{c}\mathbf{x}^{(i)} }{\sum\limits_{i=1}^{m} r^{(i)}_{c}}  = \mu^{(c)}$$

We can further substitute $N^{(c)}$ so that

$$
\frac{1}{N^{(c)}}\sum\limits_{i=1}^{m} r^{(i)}_{c}\mathbf{x}^{(i)} = \mu^{(c)}
$$

### Updating the covariances

The update of the covariance $\Sigma^{(c)}$ is given by:

$$ \Sigma^{(c)}_\text{new} = \frac{1}{N^{(c)}} \sum\limits_{i=1}^{m}r^{(i)}_{c}(\mathbf{x}^{(i)} - \mu^{(c)})(\mathbf{x}^{(i)} - \mu^{(c)})^T $$

**Proof** 

We take a partial derivative of our objective function with respect to the Sigma parameters $\Sigma^{(c)}$. Similarly, to simplify things, let's perform partial derivative without the log first and only consider one sample.

$$
	\frac{\partial p(\mathbf{x}^{(i)} | \theta)}{\partial \Sigma^{(c)}} = \frac{\partial}{\partial \Sigma^{(c)}} \big(\pi^{(c)}(2\pi)^{-\frac{n}{2}} \det(\Sigma^{(c)})^{\frac{1}{2}}e^{\big(-\frac{1}{2}(\mathbf{x}^{(i)} - \mu^{(c)})^T\Sigma^{(c)-1}(\mathbf{x}^{(i)} - \mu^{(c)})\big)\big)}
$$

Using derivative multiplication rule, we got

$$
= \pi^{(c)}(2\pi)^{-\frac{n}{2}}\big[\frac{\partial}{\partial \Sigma^{(c)}}\det(\Sigma^{(c)})^{-\frac{1}{2}}exp\big(-\frac{1}{2}(\mathbf{x}^{(i)} - \mu^{(c)})^T\Sigma^{(c)-1}(\mathbf{x}^{(i)} - \mu^{(c)})) + \det(\Sigma^{(c)})^{-\frac{1}{2}}\frac{\partial}{\partial \Sigma^{(c)}}exp\big(-\frac{1}{2}(\mathbf{x}^{(i)} - \mu^{(c)})^T\Sigma^{(c)}(\mathbf{x}^{(i)} - \mu^{(c)})\big]
$$

Using this following rule

$$
\frac{\partial}{\partial X}\det(f(x)) = \det(f(x))tr\big(f(x)^{-1}\frac{\partial f(x)}{\partial x}\big)
$$

We get that

$$
\frac{\partial}{\partial \Sigma^{(c)}}\det(\Sigma^{(c)})^{-\frac{1}{2}} = -\frac{1}{2}\det(\Sigma^{(c)})^{-\frac{1}{2}}\Sigma^{(c)-1}
$$

Using this following rule

$$
\frac{\partial a^TXb}{\partial X} = ab^T
$$

We get that

$$
\frac{\partial}{\partial \Sigma^{(c)}}(\mathbf{x}^{(i)} - \mu^{(c)})^T\Sigma^{(c)-1}(\mathbf{x}^{(i)} - \mu^{(c)}) = -\Sigma^{(c)-1}(\mathbf{x}^{(i)} - \mu^{(c)})(\mathbf{x}^{(i)} - \mu^{(c)})^T\Sigma^{(c)-1}
$$

Putting them together, we got:

$$
\frac{\partial p(\mathbf{x}^{(i)} | \theta)}{\partial \Sigma^{(c)}} = \pi^{(c)}\mathcal{N}(\mathbf{x}^{(i)} | \mu^{(c)}, \Sigma^{(c)}) * \big[-\frac{1}{2}(\Sigma^{(c)-1}-\Sigma^{(c)-1}(\mathbf{x}^{(i)}-\mu^{(c)})(\mathbf{x}^{(i)} - \mu^{(c)})^T\Sigma^{(c)-1}\big]
$$

Now consider all samples and log as well, the partial derivative of the log-likelihood with respect to $\Sigma^{(c)}$ is given by

$$
\begin{aligned}
\frac{\partial \mathcal{L}}{\partial \Sigma^{(c)}} &=  \sum\limits_{i=1}^{m}\frac{\partial \log p(\mathbf{x}^{(i)} | \theta)}{\partial \Sigma^{(c)}}\\
&=\sum\limits_{i=1}^{m}\frac{1}{(p(\mathbf{x}^{(i)} | \theta)}\frac{\partial p(\mathbf{x}^{(i)} | \theta)}{\partial \Sigma^{(c)}}\\
&=\sum\limits_{i=1}^{m}\frac{\pi^{(c)}\mathcal{N}(\mathbf{x}^{(i)} \mid \mu^{(c)}, \Sigma^{(c)})}{\Sigma_{c=1}^{k} \pi^{(c)}\mathcal{N}(\mathbf{x}^{(i)} \mid \mu^{(c)}, \Sigma^{(c)})} * \big[-\frac{1}{2}(\Sigma^{(c)-1}-\Sigma^{(c)-1}(\mathbf{x}^{(i)}-\mu^{(c)})(\mathbf{x}^{(i)} - \mu^{(c)})^T\Sigma^{(c)-1})\big]
\end{aligned}
$$

Substituting $r^{(i)}_{c}$, we got

$$
= -\frac{1}{2}\sum\limits_{i=1}^{m}r^{(i)}_{c}(\Sigma^{(c)-1}-\Sigma^{(c)-1}(\mathbf{x}^{(i)}-\mu^{(c)})(\mathbf{x}^{(i)} - \mu^{(c)})^T\Sigma^{(c)-1})\\
= -\frac{1}{2}\Sigma^{(c)-1}\sum\limits_{i=1}^{m}r^{(i)}_{c} + \frac{1}{2}\Sigma^{(c)-1}\big(\sum\limits_{i=1}^{m}r^{(i)}_{c}(\mathbf{x}^{(i)}-\mu^{(c)})(\mathbf{x}^{(i)} - \mu^{(c)})^T\big)\Sigma^{(c)-1}
$$

Setting this to zero, we obtain:

$$
N^{(c)}\Sigma^{(c)-1} = \Sigma^{(c)-1}\big(\sum\limits_{i=1}^m r^{(i)}_{c}(\mathbf{x}^{(i)}-\mu^{(c)})(\mathbf{x}^{(i)} - \mu^{(c)})^T\big)\Sigma^{(c)-1}
$$

By solving for $\Sigma^{(c)}$ we got

$$
\Sigma^{(c)} = \frac{1}{N^{(c)}}\sum\limits_{i=1}^{m}r^{(i)}_{c}(\mathbf{x}^{(i)}-\mu^{(c)})(\mathbf{x}^{(i)} - \mu^{(c)})^T
$$



### Updating the pi

The update of the mixture weights $\pi^{(c)}$:

$$ \pi^{(c)}_\text{new} = \frac{N^{(c)}}{m}$$

**Proof**

To find the partial derivative, we account for the equality constraint 

$$\sum\limits_{c=1}^k \pi^{(c)}=1$$

The Lagrangian $\mathscr{L}$ is

$$
\begin{aligned}
\mathscr{L} &= \mathcal{L} + \beta\big(\sum\limits_{c=1}^k \pi^{(c)}-1\big)\\
&= \sum\limits_{i=1}^m \log \sum\limits_{c=1}^k \pi^{(c)} \mathcal{N}(x | \mu^{(c)}, \Sigma^{(c)}) + \beta\big(\sum\limits_{c=1}^k \pi^{(c)}-1\big)
\end{aligned}
$$

Taking the partial derivative with respect to $\pi^{(c)}$ as

$$
\begin{aligned}
\frac{\partial \mathscr{L}}{\partial \pi^{(c)}} &= \sum\limits_{i=1}^m
\frac{\mathcal{N}(\mathbf{x}^{(i)} \mid \mu^{(c)}, \Sigma^{(c)})}{\Sigma_{c=1}^{k} \pi^{(c)}\mathcal{N}(\mathbf{x}^{(i)} \mid \mu^{(c)}, \Sigma^{(c)})} + \beta \\
&= \frac{1}{\pi^{(c)}}\sum\limits_{i=1}^m\frac{\pi^{(c)}\mathcal{N}(\mathbf{x}^{(i)} \mid \mu^{(c)}, \Sigma^{(c)})}{\Sigma_{c=1}^{k} \pi^{(c)}\mathcal{N}(\mathbf{x}^{(i)} \mid \mu^{(c)}, \Sigma^{(c)})} + \beta\\
&= \frac{N^{(c)}}{\pi^{(c)}} + \beta
\end{aligned}
$$

Taking the partial derivative with respect to $\beta$ is

$$
\frac{\partial \mathscr{L}}{\partial \beta} = \sum\limits_{c=1}^{k} \pi^{(c)} - 1
$$

Setting both partial derivatives to zero yield

$$
\pi^{(c)} = -\frac{N^{(c)}}{\beta}
$$

$$
1 = \sum\limits_{c=1}^k\pi^{(c)}
$$

Using the top formula to solve for the bottom formula:

$$    
- \sum\limits_{i=1}^{m}\frac{N^{(c)}}{\beta} = 1\\
= -\frac{m}{\beta} = 1\\
= \beta = -m
$$

Substitute $-m$ for $\beta$ yield

$$
\pi^{(c)} = \frac{N^{(c)}}{m}
$$

## Let's code!

### Step 1: Define k random clusters

Define k clusters from k random number of gaussian distribution.  Specifically, for each cluster $c$, randomly initialize parameters mean $\mathbf{\mu}^{(c)}$, covariance $\Sigma^{(c)}$, fraction per class $\pi^{(c)}$ and responsiblities (likelihood) of each sample $r^{(i)}_{c}$ 
    
Recall that gaussian distribution is parametrized by the mean $\mathbf{\mu}$ and the covariance $\Sigma$


### 1.1 Define means

### 1.2 Define covariance

For those who forget about covariance matrix, 

$$\text{cov}(a) = \text{var}(a) = \sum_i^m \frac{(a^i - \mu)^2}{m}$$

$$\text{cov}(a, b) = \sum_i^m \frac{(a^i - \mu)(b^i - \mu)}{m}$$

### 1.3 Define pi

### 1.4 Define responsibilities

## Step 2: EM-step

Repeat until converged:

   1. *E-step*: for each sample $\mathbf{x}^{(i)}$, evaluate responsibilities $r^{(i)}_{c}$ for every data point $\mathbf{x}^{(i)}$ using 

$$ r^{(i)}_{c} = \frac{\pi^{(c)}\mathcal{N}(\mathbf{x}^{(i)} \mid \mu^{(c)}, \Sigma^{(c)})}{\Sigma_{c=1}^{k} \pi^{(c)}\mathcal{N}(\mathbf{x}^{(i)} \mid \mu^{(c)}, \Sigma^{(c)})}$$

   2. *M-step*: for each cluster, update the gaussian distribution of each cluster, i.e., restimate parameters $\pi^{(c)}, \mu^{(c)}, \Sigma^{(c)}, N^{(c)}$ using the updated responsibilites $r^{(i)}_{c}$.

$$N^{(c)} = \sum\limits_{i=1}^{m}r^{(i)}_c$$

$$ \mu^{(c)}_\text{new} = \frac{1}{N^{(c)}} \sum\limits_{i=1}^{m}r^{(i)}_{c}\mathbf{x}^{(i)}$$

$$ \Sigma^{(c)}_\text{new} = \frac{1}{N^{(c)}} \sum\limits_{i=1}^{m}r^{(i)}_{c}(\mathbf{x}^{(i)} - \mu^{(c)})(\mathbf{x}^{(i)} - \mu^{(c)})^T$$

$$ \pi^{(c)}_\text{new} = \frac{N^{(c)}}{m}$$




### 2.1 E-Step

### 2.2 M-Step

### 2.3 Update Pi

### 2.4 Update mean

### 2.5 Update covariance

## Step 3: Predict

## Putting everything together